<a href="https://colab.research.google.com/github/shnhrtkyk/JTCcode/blob/main/05_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorchの使い方

講習では、Pytorchという深層学習ライブラリを使用してモデルの構築や学習を行います。
まずは、簡単な処理を構築してみましょう。

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# 線形変換を行う関数
linear = nn.Linear(128, 256)
# 入力データを作成
input_data = torch.zeros((128))

x = linear(input_data)
# 活性化関数を適用
x = F.relu(x)
print(x.shape)

torch.Size([256])


## 簡単なマルチレイヤーパーセプトロンの実装

マルチレイヤーパーセプトロンは全結合層で成り立つニューラルネットワークです。
Pytorchで実装する際には、まず使いたい関数を`__init__`で定義します。
その後、実際に順計算するのは`forward`の中です。


In [2]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        # 線形変換を行う全結合層を定義
        self.linear1 = nn.Linear(128, 256)
        # 線形変換を行う全結合層を定義
        self.linear2 = nn.Linear(256, 10)
        
    def forward(self, x):
        x = self.linear1(x) # まず、全結合層へ入力する
        x = F.relu(x) # 次に活性化関数を適用する
        x = self.linear2(x) # 最後に全結合層へ入力する
        return x
# MLPという名前で定義した層設計のクラスをnetという名前で呼び出します。
net = MLP()
# netにinput_dataを入力する
y = net(input_data)
print(y.shape)

torch.Size([10])


そのほかにも層設計の定義方法はあり`nn.Sequential`で同様の処理を実現できます。

In [3]:
net = nn.Sequential(nn.Linear(128, 256), nn.ReLU(), nn.Linear(256, 10))
y = net(input_data)
print(y.shape)

torch.Size([10])


また、入力する次元は、バッチと呼ばれる一度にモデルへ入力することができるひとまとまりごとに入力できます。
以下の例では、32個のデータを一気に深層学習モデルへ入力します。

In [5]:
input_data = torch.rand([32, 128])
y = net(input_data)
print(y.shape)

torch.Size([32, 10])


## 学習のやりかた

パラメータの最適化
===========================

モデルとデータを用意できたので続いてはモデルを訓練、検証することで、データに対してモデルのパラメータを最適化し、テストを行います。




モデルの訓練は反復的なプロセスとなります。

各イテレーション（エポックと呼ばれます）で、モデルは出力を計算し、損失を求めます。そして各パラメータについて損失に対する偏微分の値を求めます。

その後、勾配降下法に基づいてパラメータを最適化します。



---


## 層設計など 


In [7]:
%matplotlib inline

In [8]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

---



ハイパーパラメータ
-----------------

ハイパーパラメータは、モデルの最適化プロセスを制御するためのパラメータです。

ハイパーパラメータの値が異なると、モデルの学習や収束率に影響します（詳細なハイパーパラメータチューニングの解説は[こちら](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html)をご覧ください）。




今回は、訓練用のハイパーパラメータとして以下の値を使用します。

 - **Number of Epochs**：イテレーション回数
 - **Batch Size**：ミニバッチサイズを構成するデータ数
 - **Learning Rate**：パラメータ更新の係数。値が小さいと変化が少なく、大きすぎると訓練に失敗する可能性が生まれる
 

In [9]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

---


最適化ループ
-----------------
ハイパーパラメータを設定後、訓練で最適化のループを回すことで、モデルを最適化します。

最適化ループの1回のイテレーションは、**エポック(epoch)**と呼ばれます。

各エポックでは2種類のループから構成されます。

 - **訓練ループ**：データセットに対して訓練を実行し、パラメータを収束させます
 
 - **検証 / テストループ**：テストデータセットでモデルを評価し、性能が向上しているか確認します
 



訓練ループ内で使用される概念について、簡単に把握しておきましょう。

本チュートリアルの最後には、最適化ループの完全な実装を紹介します。

**損失関数：Loss Function**

データが与えられても、訓練されていないネットワークは正しい答えを出力しない可能性があります。
 
損失関数はモデルが推論した結果と、実際の正解との誤差の大きさを測定する関数です。訓練ではこの損失関数の値を小さくしていきます。

損失を計算するためには、入力データに対するモデルの推論結果を求め、その値と正解のラベルとの違いを比較します。



一般的な損失関数としては、回帰タスクでは[`nn.MSELoss`](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss)(Mean Square Error)、分類タスクでは[`nn.NLLLoss`](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss)(Negative Log Likelihood) が使用されます。

[`nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss)は、``nn.LogSoftmax`` と ``nn.NLLLoss``を結合した損失関数となります。

モデルが出力する`logit`値を`nn.CrossEntropyLoss`に与えて正規化し、予測誤差を求めます。

In [10]:
# loss functionの初期化、定義
loss_fn = nn.CrossEntropyLoss()

**最適化器：Optimizer**

最適化は各訓練ステップにおいてモデルの誤差を小さくなるように、モデルパラメータを調整するプロセスです。

<br>

**最適化アルゴリズム：Optimization algorithms**

最適化アルゴリズムは、最適化プロセスの具体的な手続きです（本チュートリアルでは確率的勾配降下法：Stochastic Gradient Descentを使用します）。

最適化のロジックは全て``optimizer``オブジェクト内に隠ぺいされます。



 今回はSGD optimizerを使用します。ただし、最適化関数にはADAMやRMSPropなど、様々な種類があります。
 



訓練したいモデルパラメータをoptimizerに登録し、合わせて学習率をハイパーパラメータとして渡すことで初期化を行います。




In [11]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

訓練ループ内で、最適化（optimization）は3つのステップから構成されます。

[1] ``optimizer.zero_grad()``を実行し、モデルパラメータの勾配をリセットします。

勾配の計算は蓄積されていくので、毎イテレーション、明示的にリセットします。

<br>

[2] 続いて、``loss.backwards()``を実行し、バックプロパゲーションを実行します。

PyTorchは損失に対する各パラメータの偏微分の値（勾配）を求めます。

<br>

[3] 最後に、``optimizer.step()``を実行し、各パラメータの勾配を使用してパラメータの値を調整します。



---


実装全体：Full Implementation
-----------------------
最適化を実行するコードをループする``train_loop``と、テストデータに対してモデルの性能を評価する``test_loop``を定義します。




In [12]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):        
        # 予測と損失の計算
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # バックプロパゲーション
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

損失関数とoptimizerを初期化し、それを ``train_loop`` と ``test_loop`` に渡します。







In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.298138  [    0/60000]
loss: 2.294968  [ 6400/60000]
loss: 2.277383  [12800/60000]
loss: 2.285448  [19200/60000]
loss: 2.280416  [25600/60000]
loss: 2.242236  [32000/60000]
loss: 2.273299  [38400/60000]
loss: 2.239137  [44800/60000]
loss: 2.252436  [51200/60000]
loss: 2.231144  [57600/60000]
Test Error: 
 Accuracy: 26.5%, Avg loss: 0.034983 

Epoch 2
-------------------------------
loss: 2.225547  [    0/60000]
loss: 2.226835  [ 6400/60000]
loss: 2.178338  [12800/60000]
loss: 2.214661  [19200/60000]
loss: 2.201105  [25600/60000]
loss: 2.128296  [32000/60000]
loss: 2.204439  [38400/60000]
loss: 2.131501  [44800/60000]
loss: 2.170639  [51200/60000]
loss: 2.139056  [57600/60000]
Test Error: 
 Accuracy: 35.6%, Avg loss: 0.033286 

Epoch 3
-------------------------------
loss: 2.117138  [    0/60000]
loss: 2.111336  [ 6400/60000]
loss: 2.015203  [12800/60000]
loss: 2.103530  [19200/60000]
loss: 2.064550  [25600/60000]
loss: 1.952058  [32000/600